# Deserializing by Field Name or Alias

In [6]:
from pydantic import ConfigDict, Field, BaseModel, ValidationError


class Model(BaseModel):
    model_config = ConfigDict(populate_by_name=True)

    id_: int = Field(alias="id")
    first_name: str = Field(alias="firstName")

And now we can use **either** the field name or the alias:

In [7]:
Model(id_=10, first_name="Newton")

Model(id_=10, first_name='Newton')

We can even mix it up:

In [8]:
Model(id_=10, firstName="Newton")

Model(id_=10, first_name='Newton')

And this works with dict or JSON deserialization too:

In [9]:
data = {
    "id_": 10,
    "first_name": "Newton"
}

Model.model_validate(data)

Model(id_=10, first_name='Newton')

To recap what we have so far for configuring aliases and associated behaviors.

In [11]:
from pydantic.alias_generators import to_camel


class Person(BaseModel):
    model_config = ConfigDict(
        alias_generator=to_camel,
        populate_by_name=True,
        extra="forbid"
    )

    id_: int = Field(alias="id", default=1)
    first_name: str | None = None
    last_name: str
    age: int | None = None

In [12]:
p = Person(id=10, first_name='Isaac', lastName='Newton', age=84)
p

Person(id_=10, first_name='Isaac', last_name='Newton', age=84)

In [11]:
data_json = """
{
    "id": 10,
    "firstName": "Isaac",
    "last_name": "Newton",
    "age": 84
}
"""

p = Person.model_validate_json(data_json)
p

Person(id_=10, first_name='Isaac', last_name='Newton', age=84)

In [12]:
p.model_dump()

{'id_': 10, 'first_name': 'Isaac', 'last_name': 'Newton', 'age': 84}

Notice that the `populate_by_name` setting does not affect the names used when deserializing - the default is still to use the field name, not the alias.

We need to specifically request this:

In [13]:
p.model_dump(by_alias=True)

{'id': 10, 'firstName': 'Isaac', 'lastName': 'Newton', 'age': 84}

In [14]:
p.model_dump_json(by_alias=True)

'{"id":10,"firstName":"Isaac","lastName":"Newton","age":84}'